In [1]:
import pandas as pd
import plotly.graph_objects as go
import sqlite3

# Connect to the database
conn = sqlite3.connect('../../../data/stocks_analysis.db')

# Query the combined filtered data from the database
combined_filtered_data = pd.read_sql_query("SELECT * FROM mean_results", conn)

# Disconnect from the database
conn.close()

# Available columns (metrics) to plot
available_columns = ['Percentage Change (%)', 'Volume', 'Volatility', 'Quarter ROI (%)']

# Set default metric
metric = 'Percentage Change (%)'

# Calculate the mean for Oil and Telecom sectors
mean_data = combined_filtered_data.groupby(['Quarter', 'Sector']).mean().reset_index()

In [2]:
# Create an empty figure object
fig = go.Figure()

# Add bar traces for each Sector using the default metric
for Sector in combined_filtered_data['Sector'].unique():
    Sector_data = combined_filtered_data[combined_filtered_data['Sector'] == Sector]
    fig.add_trace(go.Bar(x=Sector_data['Quarter'], y=Sector_data[metric], name=Sector))

# Create dropdown menu for selecting different metrics
dropdown_buttons = [
    dict(
        label=col,
        method="update",
        args=[{"y": [combined_filtered_data[combined_filtered_data['Sector'] == Sector][col] for Sector in combined_filtered_data['Sector'].unique()]}]
    ) for col in available_columns
]

# Create layout with dropdown
fig.update_layout(
    xaxis_title="Quarter",
    legend=dict(
        orientation="v",
        y=0.5,
        x=1.1
    ),
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction="down",
            showactive=True,
            x=0.17,  # Adjust the position of the dropdown
            xanchor="left",
            y=1.2,
            yanchor="top"
        )
    ],
    barmode='group'  # This ensures the bars for each sector are grouped by Quarter
)

# Display the plot
fig.show()

In [3]:
# Save the telecom plot as an HTML file
fig.write_html('mean_results_plotly.html')
